## Hashing trick for look-alike portal application completed


In [2]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
#import numpy as np
#import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
try:
    sc.stop()
except NameError: pass


conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "20g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

import pandas as pd
import numpy as np
import os
import sklearn
import re
import xgboost as xgb

/home/k.p.osminin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Данные в формате libsvm. Хешированные фрагменты урлов как факторы с количестом посещений в качестве значения. Первое поле - метка класса. Класс положителен, если он оставил полностью заполненную заявку на портале в течение следующих 3 дней.

In [2]:

# read in data
dtrain = xgb.DMatrix('/home/k.p.osminin/external_hdfs/ha_proc_20160929.txt')
dtest = xgb.DMatrix('/home/k.p.osminin/external_hdfs/ha_proc_20161005.txt')
# specify parameters via map

In [3]:
param = {'max_depth':10, 'eta':1, 'gamma' : 0.5, 'silent':1, 'objective':'binary:logistic','eval_metric' : "auc",
         'scale_pos_weight' : 1, 'num_round' : 10 }

bst = xgb.train(param, dtrain)

# make prediction
preds = bst.predict(dtest)

In [4]:

print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = dtest.get_label(), 
                  y_score = bst.predict(dtest)
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = dtrain.get_label(), 
                  y_score = bst.predict(dtrain)
                )                                              
))

XGBoost AUCROC test: 0.795107583536. train 0.821805099016


In [ ]:
param2 = {'max_depth':10, 'eta':1, 'gamma' : 0.5, 'silent':1, 'objective':'binary:logistic','eval_metric' : "auc",
         'scale_pos_weight' : 1, 'num_round' : 2 }
import itertools
import numpy as np
param_grid = [
    ('max_depth',[3,5,8,10,15]), 
    ('eta',[0.05,0.1,0.5,1]), 
    ('gamma' , [0.1,0.5,1,2]), 
    ('objective',['binary:logistic']),
    ('eval_metric' , ['auc']),
    ('scale_pos_weight' , [1,2,5,10]), 
    ('num_round' , [4,10,20]),
]

param_grid_values = list(itertools.product(*[e[1] for e in param_grid]))

dice_num = 30

for i in np.random.choice(range(len(param_grid_values)),dice_num, replace = False):
    param = zip([e[0] for e in param_grid],param_grid_values[i])
    #print(param)
    bst = xgb.train(param, dtrain)
    print('{0} AUCROC {1}. {2}'.format(
               param, 
               bst.eval(dtest, name='test'),
               bst.eval(dtrain, name='train'),
    ))

выбирается вариант [('max_depth', 8), ('eta', 1), ('gamma', 2), ('objective', 'binary:logistic'), ('eval_metric', 'auc'), ('scale_pos_weight', 1), ('num_round', 4)] AUCROC [0]	test-auc:0.793204. [0]	train-auc:0.813929

In [ ]:
param = [('max_depth', 10), ('eta', 0.8), ('gamma', 1), ('objective', 'binary:logistic'), ('eval_metric', 'auc'), 
         ('scale_pos_weight', 1), ('num_round', 10)]

files_list = sorted([e for e in os.popen("ls -dX /home/k.p.osminin/external_hdfs/* | grep -i 'ha_proc_'").read().split('\n') if e])
test_file = '/home/k.p.osminin/external_hdfs/ha_proc_sampled_20161010.txt'
dtest = xgb.DMatrix(test_file)
bst = xgb.Booster()

for file_name in [f for f in files_list if not re.findall('20161010',f)]:
    dtrain = xgb.DMatrix(file_name)
    bst = xgb.train(param, dtrain,xgb_model = bst)
    print('{0} AUCROC {1}. {2}'.format(
               re.findall('[0-9]+',file_name)[0], 
               bst.eval(dtest, name='test'),
               bst.eval(dtrain, name='train'),
    ))


In [ ]:
param = [('max_depth', 10), ('eta', 0.8), ('gamma', 1), ('objective', 'binary:logistic'), ('eval_metric', 'auc'), 
         ('scale_pos_weight', 1), ('num_round', 10)]

files_list = sorted([e for e in os.popen("ls -dX /home/k.p.osminin/external_hdfs/* | grep -i 'ha_proc_'").read().split('\n') if e])
test_file = '/home/k.p.osminin/external_hdfs/ha_proc_sampled_20161010.txt'
dtest = xgb.DMatrix(test_file)
bst = xgb.Booster()


In [ ]:
param = {'max_depth':10, 'eta':1, 'gamma' : 0.5, 'silent':1, 'objective':'binary:logistic','eval_metric' : "auc",
         'scale_pos_weight' : 1, 'num_round' : 10 }

files_list = sorted([e for e in os.popen("ls -dX /home/k.p.osminin/external_hdfs/* | grep -i 'ha_proc_'").read().split('\n') if e])
test_file = '/home/k.p.osminin/external_hdfs/ha_proc_sampled_20161010.txt'
#dtest = xgb.DMatrix(test_file)
bst1 = xgb.Booster()

file_name = files_list[10]
dtrain = xgb.DMatrix(file_name)
bst2 = xgb.train(param,dtrain,xgb_model = bst1)

In [4]:
files_list = sorted([e for e in os.popen("ls -dX /home/k.p.osminin/external_hdfs/* | grep -i 'ha_proc_'").read().split('\n') if e])
files_list

['/home/k.p.osminin/external_hdfs/ha_proc_20160915.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160916.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160917.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160918.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160919.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160920.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160921.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160922.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160923.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160924.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160929.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20160930.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20161001.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20161002.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20161003.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20161004.txt',
 '/home/k.p.osminin/external_hdfs/ha_proc_20161005.txt',
 '/home/k.p.osminin/external_hd